# Collecting time weighted results

> This is a more advanced topic related to results collection in DES.

Previously we calculated utilisation of servers by tracking the total usage time of resources (e.g. Nurses).  At the end of a run of the model this total was divided by the total scheduled time for that resource (i.e. no. resources multipled by the results collection period).  A downside to this approach is that when we include a warm-up period in the model there is a minor adjustment we need to make to results to account for servers already being in use. If we don't make this adjustment we can end up with utilisations higher than 100% we carry forward some resource usage time from the warm-up period into the results collection period.

Instead of modifying the model logic we could instead calculate a **time weighted average**.  We will do that by subclassing the `simpy.Resource` class to create a `MonitoredResource`.  Each time a `MonitoredResource` is requested or released we will update some statistics.

This concept is often presented in a complex way. But in general you can think of another approach to tracking.  In the modified code below we will track:

1. The time of the last request or release event for the resource.
2. The number of servers in use **before** the event executes.

This data allows us to calculate a running total of the time the total time resources have been in use up to that time point. When we update the total we:

* multiply the number of servers in use **before** the event by the time elapsed since the last event.
* Add this value to the running total.

At the end of the run we can compute utilisation by dividing the running total by the total scheduled time.

## 1. Imports

In [1]:
import numpy as np
import pandas as pd
import simpy
import itertools

## 2. Notebook level variables, constants, and default values

A useful first step when setting up a simulation model is to define the base case or as-is parameters.  Here we will create a set of constant/default values for our `Experiment` class, but you could also consider reading these in from a file.

In [2]:
# default resources
N_OPERATORS = 13

# ##############################################################################
# MODIFICATION: number of nurses available
N_NURSES = 10
# ##############################################################################

# default mean inter-arrival time (exp)
MEAN_IAT = 60 / 100

## default service time parameters (triangular)
CALL_LOW = 5.0
CALL_MODE = 7.0
CALL_HIGH = 10.0

# ##############################################################################
# MODIFICATION: nurse defaults

# nurse uniform distribution parameters
NURSE_CALL_LOW = 10.0
NURSE_CALL_HIGH = 20.0

# probability of a callback (parameter of Bernoulli)
CHANCE_CALLBACK = 0.4

# sampling settings - we now need 4 streams
N_STREAMS = 4
DEFAULT_RND_SET = 0
# ##############################################################################

# Boolean switch to simulation results as the model runs
TRACE = False

# run variables
RESULTS_COLLECTION_PERIOD = 1000

# ##############################################################################
# MODIFICATON: added a warm-up period, by default we will not use it.
WARM_UP_PERIOD = 0
# ##############################################################################

## A Monitored Resource

In [3]:
class MonitoredResource(simpy.Resource):
    '''
    Subclass of simpy.Resource. 

    Based on method described in Law. Simulation Modeling and Analysis 4th Ed.
    Pages 14 - 17

    Calculates both resource utilisation and number in queue.
    
    '''
    def __init__(self, *args, **kwargs):
        # super() is the super class i.e. simpy.Resource 
        super().__init__(*args, **kwargs)
        # the time of the last request or release
        self.init_results()

    def init_results(self):

        self.time_last_event = self._env.now
        
        # the running total of the area under the no. in queue function.
        self.area_n_in_queue = 0.0
        
        # the running total of the area under the server busy function.
        self.area_resource_busy = 0.0
        
    def request(self, *args, **kwargs):
        # update time weighted stats BEFORE requesting resource.
        self.update_time_weighted_stats()
        return super().request(*args, **kwargs)

    def release(self, *args, **kwargs):
        # update time weighted stats BEFORE releasing resource.
        self.update_time_weighted_stats()
        return super().release(*args, **kwargs)

    def update_time_weighted_stats(self):
        # time since the last release/request
        time_since_last_event = self._env.now - self.time_last_event

        # update last event time
        self.time_last_event = self._env.now

        # update the area under the no. in queue function.
        # len(self.queue) is the number of requests queued.
        self.area_n_in_queue += len(self.queue) * time_since_last_event
        
        # update the area under the resource busy function.
        # self.count is the number of resources in use.
        self.area_resource_busy += self.count * time_since_last_event

    def end_of_run_cleanup(self, run_length):
        yield self._env.timeout(run_length)

        # update time weighted stats - adds in uncounted resource usage.
        # from last event time until end of simulation run.
        self.update_time_weighted_stats()

## 3. Distribution classes

We will define two additional distribution classes (`Uniform` and `Bernoulli`) to encapsulate the random number generation, parameters and random seeds used in the sampling.  Take a look at how they work.

> You should be able to reuse these classes in your own simulation models.  It is actually not a lot of code, but it is useful to build up a code base that you can reuse with confidence in your own projects.

In [4]:
class Bernoulli:
    """
    Convenience class for the Bernoulli distribution.
    packages up distribution parameters, seed and random generator.

    The Bernoulli distribution is a special case of the binomial distribution
    where a single trial is conducted

    Use the Bernoulli distribution to sample success or failure.
    """

    def __init__(self, p, random_seed=None):
        """
        Constructor

        Params:
        ------
        p: float
            probability of drawing a 1

        random_seed: int | SeedSequence, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        """
        self.rand = np.random.default_rng(seed=random_seed)
        self.p = p

    def sample(self, size=None):
        """
        Generate a sample from the exponential distribution

        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.

        Returns:
        -------
        float or np.ndarray (if size >=1)
        """
        return self.rand.binomial(n=1, p=self.p, size=size)

In [5]:
class Uniform:
    """
    Convenience class for the Uniform distribution.
    packages up distribution parameters, seed and random generator.
    """

    def __init__(self, low, high, random_seed=None):
        """
        Constructor

        Params:
        ------
        low: float
            lower range of the uniform

        high: float
            upper range of the uniform

        random_seed: int | SeedSequence, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        """
        self.rand = np.random.default_rng(seed=random_seed)
        self.low = low
        self.high = high

    def sample(self, size=None):
        """
        Generate a sample from the exponential distribution

        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.

        Returns:
        -------
        float or np.ndarray (if size >=1)
        """
        return self.rand.uniform(low=self.low, high=self.high, size=size)

In [6]:
class Triangular:
    """
    Convenience class for the triangular distribution.
    packages up distribution parameters, seed and random generator.
    """

    def __init__(self, low, mode, high, random_seed=None):
        """
        Constructor. Accepts and stores parameters of the triangular dist
        and a random seed.

        Params:
        ------
        low: float
            The smallest values that can be sampled

        mode: float
            The most frequently sample value

        high: float
            The highest value that can be sampled

        random_seed: int | SeedSequence, optional (default=None)
            Used with params to create a series of repeatable samples.
        """
        self.rand = np.random.default_rng(seed=random_seed)
        self.low = low
        self.high = high
        self.mode = mode

    def sample(self, size=None):
        """
        Generate one or more samples from the triangular distribution

        Params:
        --------
        size: int
            the number of samples to return.  If size=None then a single
            sample is returned.

        Returns:
        -------
        float or np.ndarray (if size >=1)
        """
        return self.rand.triangular(self.low, self.mode, self.high, size=size)

In [7]:
class Exponential:
    """
    Convenience class for the exponential distribution.
    packages up distribution parameters, seed and random generator.
    """

    def __init__(self, mean, random_seed=None):
        """
        Constructor

        Params:
        ------
        mean: float
            The mean of the exponential distribution

        random_seed: int| SeedSequence, optional (default=None)
            A random seed to reproduce samples.  If set to none then a unique
            sample is created.
        """
        self.rand = np.random.default_rng(seed=random_seed)
        self.mean = mean

    def sample(self, size=None):
        """
        Generate a sample from the exponential distribution

        Params:
        -------
        size: int, optional (default=None)
            the number of samples to return.  If size=None then a single
            sample is returned.

        Returns:
        -------
        float or np.ndarray (if size >=1)
        """
        return self.rand.exponential(self.mean, size=size)

## 3. Experiment class

We will modify the experiment class to include new results collection for the additional nurse process. 

1. Modify the __init__ method to accept additional parameters: `chance_callback`, `nurse_call_low`, `nurse_call_high`. Remember to include the default values for these parameters.
2. Store parameters in the class and create new distributions.
3. Add variables to support KPI calculation to the `results` dictionary for `nurse_waiting_times` and `total_nurse_call_duration`

In [8]:
class Experiment:
    """
    Encapsulates the concept of an experiment 🧪 with the urgent care
    call centre simulation model.

    An Experiment:
    1. Contains a list of parameters that can be left as defaults or varied
    2. Provides a place for the experimentor to record results of a run
    3. Controls the set & streams of psuedo random numbers used in a run.

    """

    def __init__(
        self,
        random_number_set=DEFAULT_RND_SET,
        n_streams=N_STREAMS,
        n_operators=N_OPERATORS,
        mean_iat=MEAN_IAT,
        call_low=CALL_LOW,
        call_mode=CALL_MODE,
        call_high=CALL_HIGH,
        # ######################################################################
        # MODIFICATION: nurse parameters
        n_nurses=N_NURSES,
        chance_callback=CHANCE_CALLBACK,
        nurse_call_low=NURSE_CALL_LOW,
        nurse_call_high=NURSE_CALL_HIGH,
        ########################################################################
    ):
        """
        The init method sets up our defaults.
        """
        # sampling
        self.random_number_set = random_number_set
        self.n_streams = n_streams

        # store parameters for the run of the model
        self.n_operators = n_operators
        self.mean_iat = mean_iat
        self.call_low = call_low
        self.call_mode = call_mode
        self.call_high = call_high

        # resources: we must init resources after an Environment is created.
        # But we will store a placeholder for transparency
        self.operators = None

        # ######################################################################
        # MODIFICATION: nurse parameters
        self.n_nurses = n_nurses
        self.chance_callback = chance_callback
        self.nurse_call_low = nurse_call_low
        self.nurse_call_high = nurse_call_high

        # nurse resources placeholder
        self.nurses = None
        # ######################################################################

        # initialise results to zero
        self.init_results_variables()

        # initialise sampling objects
        self.init_sampling()

    def set_random_no_set(self, random_number_set):
        """
        Controls the random sampling
        Parameters:
        ----------
        random_number_set: int
            Used to control the set of pseudo random numbers used by
            the distributions in the simulation.
        """
        self.random_number_set = random_number_set
        self.init_sampling()

    def init_sampling(self):
        """
        Create the distributions used by the model and initialise
        the random seeds of each.
        """
        # produce n non-overlapping streams
        seed_sequence = np.random.SeedSequence(self.random_number_set)
        self.seeds = seed_sequence.spawn(self.n_streams)

        # create distributions

        # call inter-arrival times
        self.arrival_dist = Exponential(
            self.mean_iat, random_seed=self.seeds[0]
        )

        # duration of call triage
        self.call_dist = Triangular(
            self.call_low,
            self.call_mode,
            self.call_high,
            random_seed=self.seeds[1],
        )

        # ######################################################################
        # MODIFICATION create the callback and nurse consultation distributions
        self.callback_dist = Bernoulli(
            self.chance_callback, random_seed=self.seeds[2]
        )

        self.nurse_dist = Uniform(
            self.nurse_call_low,
            self.nurse_call_high,
            random_seed=self.seeds[3],
        )
        # ######################################################################

    def init_results_variables(self):
        """
        Initialise all of the experiment variables used in results
        collection.  This method is called at the start of each run
        of the model
        """
        # variable used to store results of experiment
        self.results = {}
        self.results["waiting_times"] = []

        # total operator usage time for utilisation calculation.
        self.results["total_call_duration"] = 0.0

        # ######################################################################
        # nurse sub process results collection
        self.results["nurse_waiting_times"] = []
        self.results["total_nurse_call_duration"] = 0.0
        # ######################################################################

        # ######################################################################
        # MODIFICATIONS: reset results collected in montiored resources.
        if self.operators is not None:
            self.operators.init_results()

        if self.nurses is not None:
            self.nurses.init_results()

## 4. Modified model code

We will modify the model code and logic that we have already developed to include a nurse consultation for a proportion of the callers.  We create a new function called `nurse_consultation` that contains all the logic. We also need to modify the `service` function so that a proportion of calls are sent to the nurse consultation process.  

In [9]:
def trace(msg):
    """
    Turing printing of events on and off.

    Params:
    -------
    msg: str
        string to print to screen.
    """
    if TRACE:
        print(msg)

In [10]:
def nurse_consultation(identifier, env, args):
    """
    simulates the wait for an consultation with a nurse on the phone.

    1. request and wait for a nurse resource
    2. phone consultation (uniform)
    3. release nurse and exit system

    """
    trace(f"Patient {identifier} waiting for nurse call back")
    start_nurse_wait = env.now

    # request a nurse
    with args.nurses.request() as req:
        yield req

        # record the waiting time for nurse call back
        nurse_waiting_time = env.now - start_nurse_wait
        args.results["nurse_waiting_times"].append(nurse_waiting_time)

        # sample nurse the duration of the nurse consultation
        nurse_call_duration = args.nurse_dist.sample()

        trace(f"nurse called back patient {identifier} at " + f"{env.now:.3f}")

        # schedule process to begin again after call duration
        yield env.timeout(nurse_call_duration)

        args.results["total_nurse_call_duration"] += nurse_call_duration

        trace(
            f"nurse consultation for {identifier}"
            + f" competed at {env.now:.3f}"
        )

In [11]:
def service(identifier, env, args):
    """
    simulates the service process for a call operator

    1. request and wait for a call operator
    2. phone triage (triangular)
    3. release call operator
    4. a proportion of call continue to nurse consultation

    Params:
    ------
    identifier: int
        A unique identifer for this caller

    env: simpy.Environment
        The current environent the simulation is running in
        We use this to pause and restart the process after a delay.

    args: Experiment
        The settings and input parameters for the current experiment

    """

    # record the time that call entered the queue
    start_wait = env.now

    # request an operator - stored in the Experiment
    with args.operators.request() as req:
        yield req

        # record the waiting time for call to be answered
        waiting_time = env.now - start_wait

        # store the results for an experiment
        args.results["waiting_times"].append(waiting_time)
        trace(f"operator answered call {identifier} at " + f"{env.now:.3f}")

        # the sample distribution is defined by the experiment.
        call_duration = args.call_dist.sample()

        # schedule process to begin again after call_duration
        yield env.timeout(call_duration)

        # update the total call_duration
        args.results["total_call_duration"] += call_duration

        # print out information for patient.
        trace(
            f"call {identifier} ended {env.now:.3f}; "
            + f"waiting time was {waiting_time:.3f}"
        )

    # ##########################################################################
    # MODIFICATION NURSE CALL BACK
    # does nurse need to call back?
    # Note the level of the indented code.
    callback_patient = args.callback_dist.sample()

    if callback_patient:
        env.process(nurse_consultation(identifier, env, args))
    # ##########################################################################

In [12]:
def arrivals_generator(env, args):
    """
    IAT is exponentially distributed

    Parameters:
    ------
    env: simpy.Environment
        The simpy environment for the simulation

    args: Experiment
        The settings and input parameters for the simulation.
    """
    # use itertools as it provides an infinite loop
    # with a counter variable that we can use for unique Ids
    for caller_count in itertools.count(start=1):

        # rhe sample distribution is defined by the experiment.
        inter_arrival_time = args.arrival_dist.sample()
        yield env.timeout(inter_arrival_time)

        trace(f"call arrives at: {env.now:.3f}")

        # create a service process
        env.process(service(caller_count, env, args))

## 🥵 Warm-up period

The call centre model starts from empty.  If the call centre runs 24/7 then it is a non-terminating system and our estimates of waiting time and server utilisation are biased due to the empty period at the start of the simulation.  We can remove this initialisation bias using a warm-up period.  

We will implement a warm-up through an **event** that happens once in a single run of the model.  The model will be run for the **warm-up period + results collection period**.  At the end of the warm-up period an event will happen where all variables in the current experiment are reset (e.g. empty lists and set quantitative values to 0.0).

> **Note**: at the point results are reset there are likely resources (call operators and nurses) in use. The result is that we carry over some of the resource usage time from the warm-up to results collection period. It isn't a big deal, but there's potential for resource usage time to be slightly higher than the time scheduled.


In [13]:
def warmup_complete(warm_up_period, env, args):
    """
    End of warm-up period event. Used to reset results collection variables.

    Parameters:
    ----------
    warm_up_period: float
        Duration of warm-up period in simultion time units

    env: simpy.Environment
        The simpy environment

    args: Experiment
        The simulation experiment that contains the results being collected.
    """
    yield env.timeout(warm_up_period)
    trace(f"{env.now:.2f}: Warm up complete.")
    
    args.init_results_variables()

## 5. Model wrapper functions

Modifications to make to the `single_run` function:

1. Add a warm-up parameters called `wu_period`
1. Create and the nurses resource to the experiment
2. Schedule the `warm_up_complete` process.
3. After the simulation is complete calculate the mean waiting time and mean utilisation for nurses.

In [14]:
def single_run(
    experiment, 
    rep=0,
    wu_period=WARM_UP_PERIOD, 
    rc_period=RESULTS_COLLECTION_PERIOD
):
    """
    Perform a single run of the model and return the results

    Parameters:
    -----------

    experiment: Experiment
        The experiment/paramaters to use with model

    rep: int
        The replication number.

    wu_period: float, optional (default=WARM_UP_PERIOD)
        The initial transient period of the simulation
        Results from this period are removed from final computations.

    rc_period: float, optional (default=RESULTS_COLLECTION_PERIOD)
        The run length of the model following warm up where results are
        collected.
    """

    # results dictionary.  Each KPI is a new entry.
    run_results = {}

    # reset all results variables to zero and empty
    experiment.init_results_variables()

    # set random number set to the replication no.
    # this controls sampling for the run.
    experiment.set_random_no_set(rep)

    # environment is (re)created inside single run
    env = simpy.Environment() 

    # #########################################################################
    # MODIFICATION: create the MONITORED resources
    experiment.operators = MonitoredResource(env, experiment.n_operators)
    experiment.nurses = MonitoredResource(env, experiment.n_nurses)
    # #########################################################################

    # we pass the experiment to the arrivals generator
    env.process(arrivals_generator(env, experiment))

    # add warm-up period event
    env.process(warmup_complete(wu_period, env, experiment))

    # #########################################################################            
    # MODIFICATION: clean up resources to add in any final resource usage time
    env.process(experiment.operators.end_of_run_cleanup(wu_period + rc_period))
    env.process(experiment.nurses.end_of_run_cleanup(wu_period + rc_period))
    # #########################################################################

    # run for warm-up + results collection period
    env.run(until=wu_period + rc_period)

    # end of run results: calculate mean waiting time
    run_results["01_mean_waiting_time"] = np.mean(
        experiment.results["waiting_times"]
    )

    # end of run results: calculate mean operator utilisation
    run_results["02_operator_util"] = (
        experiment.results["total_call_duration"]
        / (rc_period * experiment.n_operators)
    ) * 100.0
    
    # summary results for nurse process

    # end of run results: nurse waiting time
    run_results["03_mean_nurse_waiting_time"] = np.mean(
        experiment.results["nurse_waiting_times"]
    )

    # end of run results: calculate mean nurse utilisation
    run_results["04_nurse_util"] = (
        experiment.results["total_nurse_call_duration"]
        / (rc_period * experiment.n_nurses)
    ) * 100.0

    # ##########################################################################
    # MODIFICATION: end of run results: calculate mean operator utilisation
    # from montiored resource stats
    run_results["02a_operator_util_time_weighted"] = (
        experiment.operators.area_resource_busy
        / (rc_period * experiment.n_operators)
    ) * 100.0

    run_results["04a_nurse_util_time_weighted"] = (
        experiment.nurses.area_resource_busy
        / (rc_period * experiment.n_nurses)
    ) * 100.0

    # mean queue lengths
    run_results["05_operator_queue_length"] = (
        experiment.operators.area_n_in_queue / rc_period
    )

    run_results["06_nurse_queue_length"] = (
        experiment.nurses.area_n_in_queue / rc_period
    )
    
    # ##########################################################################

    # return the results from the run of the model
    return run_results

In [15]:
def multiple_replications(
    experiment,
    wu_period=WARM_UP_PERIOD,
    rc_period=RESULTS_COLLECTION_PERIOD,
    n_reps=5,
):
    """
    Perform multiple replications of the model.

    Params:
    ------
    experiment: Experiment
        The experiment/paramaters to use with model

    rc_period: float, optional (default=DEFAULT_RESULTS_COLLECTION_PERIOD)
        results collection period.
        the number of minutes to run the model to collect results

    n_reps: int, optional (default=5)
        Number of independent replications to run.

    Returns:
    --------
    pandas.DataFrame
    """

    # loop over single run to generate results dicts in a python list.
    results = [
        single_run(experiment, rep, wu_period, rc_period) 
        for rep in range(n_reps)
    ]

    # format and return results in a dataframe
    df_results = pd.DataFrame(results)
    df_results.index = np.arange(1, len(df_results) + 1)
    df_results.index.name = "rep"
    return df_results

In [16]:
# results with no warm-up period. 
# set this up so utilisation is close to 100%
# Notice there is a slight difference in utilisation due to floating point calcs
scenario = Experiment(n_nurses=8)
results = multiple_replications(scenario, wu_period=0)
results.describe().round(1).T.sort_index()

,count,mean,std,min,25%,50%,75%,max
01_mean_waiting_time,5.0,3.0,0.9,1.5,3.1,3.2,3.5,3.9
02_operator_util,5.0,93.0,1.3,90.9,92.9,93.5,93.6,94.2
02a_operator_util_time_weighted,5.0,93.3,1.2,91.4,93.2,93.8,93.9,94.5
03_mean_nurse_waiting_time,5.0,95.8,32.1,61.8,71.3,86.9,128.5,130.7
04_nurse_util,5.0,97.7,0.4,97.2,97.3,97.8,97.9,98.2
04a_nurse_util_time_weighted,5.0,98.4,0.2,98.0,98.3,98.4,98.6,98.6
05_operator_queue_length,5.0,5.1,1.5,2.5,5.1,5.3,6.0,6.4
06_nurse_queue_length,5.0,63.2,21.8,39.2,47.5,58.6,79.8,91.1


In [18]:
# no need to adjust output measures after warm-up reset when time weighted.
# note that the original util measures can exceeed 100% in 
# some replications as we didn't add logic to adjust.
# simulation logic remains the same. We just reset at warm-up complete.
scenario = Experiment(n_nurses=8)
results = multiple_replications(scenario, wu_period=1000)
results.describe().round(1).T.sort_index()

,count,mean,std,min,25%,50%,75%,max
01_mean_waiting_time,5.0,3.5,1.1,2.3,2.7,3.2,4.4,4.9
02_operator_util,5.0,94.9,1.5,92.3,95.0,95.1,96.0,96.1
02a_operator_util_time_weighted,5.0,94.9,1.5,92.5,94.8,95.0,95.9,96.1
03_mean_nurse_waiting_time,5.0,303.6,43.3,245.4,287.8,307.1,312.5,365.1
04_nurse_util,5.0,100.0,0.2,99.8,99.8,99.8,100.1,100.3
04a_nurse_util_time_weighted,5.0,99.9,0.1,99.8,99.9,99.9,99.9,100.0
05_operator_queue_length,5.0,5.9,1.9,3.9,4.3,5.5,7.4,8.3
06_nurse_queue_length,5.0,206.5,29.6,167.5,194.8,205.9,216.2,248.3


In [20]:
scenario = Experiment(n_nurses=11)
results = multiple_replications(scenario, wu_period=2000)
results.describe().round(1).T.sort_index()

,count,mean,std,min,25%,50%,75%,max
01_mean_waiting_time,5.0,3.7,0.8,3.0,3.1,3.5,3.9,5.0
02_operator_util,5.0,94.1,1.6,91.7,93.6,94.3,94.6,96.1
02a_operator_util_time_weighted,5.0,94.1,1.6,91.7,93.7,94.3,94.7,96.1
03_mean_nurse_waiting_time,5.0,3.8,2.0,2.1,2.5,2.8,5.7,6.3
04_nurse_util,5.0,90.6,1.5,88.3,90.3,91.0,91.6,92.0
04a_nurse_util_time_weighted,5.0,90.7,1.5,88.1,90.4,91.4,91.4,92.0
05_operator_queue_length,5.0,6.2,1.4,5.1,5.2,6.0,6.3,8.5
06_nurse_queue_length,5.0,2.6,1.3,1.3,1.6,1.9,3.8,4.2
